In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('uber.csv')

In [3]:
df.head()

,Unnamed: 0,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,24238194,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,27835199,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,44984355,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,25894730,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,17610152,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [4]:
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df.head()

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
0,2015-05-07 19:52:06.0000003,7.5,2015-05-07 19:52:06 UTC,-73.999817,40.738354,-73.999512,40.723217,1
1,2009-07-17 20:04:56.0000002,7.7,2009-07-17 20:04:56 UTC,-73.994355,40.728225,-73.994710,40.750325,1
2,2009-08-24 21:45:00.00000061,12.9,2009-08-24 21:45:00 UTC,-74.005043,40.740770,-73.962565,40.772647,1
3,2009-06-26 08:22:21.0000001,5.3,2009-06-26 08:22:21 UTC,-73.976124,40.790844,-73.965316,40.803349,3
4,2014-08-28 17:47:00.000000188,16.0,2014-08-28 17:47:00 UTC,-73.925023,40.744085,-73.973082,40.761247,5


In [6]:
df.isna().sum()

key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64

In [7]:
df[df['dropoff_latitude'].isna()]

,key,fare_amount,pickup_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count
87946,2013-07-02 03:51:57.0000001,24.1,2013-07-02 03:51:57 UTC,-73.950581,40.779692,NaN,NaN,0


In [8]:
df = df.dropna()

In [9]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [10]:
import pandas as pd
import numpy as np
import math

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0)**2
    c = 2 * np.arcsin(np.sqrt(a))
    
    # Radius of Earth in kilometers. Use 3956 for miles.
    r = 6371
    
    # Calculate the result
    return c * r

# Sample data
data = {
    'pickup_latitude': df['pickup_latitude'],
    'pickup_longitude': df['pickup_longitude'],
    'dropoff_latitude': df['dropoff_latitude'],
    'dropoff_longitude': df['dropoff_longitude']
}

# Creating a DataFrame
dist = pd.DataFrame(data)

# Calculate distance for each row
dist['distance_km'] = dist.apply(lambda row: haversine(row['pickup_latitude'], row['pickup_longitude'],
                                                   row['dropoff_latitude'], row['dropoff_longitude']), axis=1)

print(dist)

        pickup_latitude  pickup_longitude  dropoff_latitude  \

0             40.738354        -73.999817         40.723217   

1             40.728225        -73.994355         40.750325   

2             40.740770        -74.005043         40.772647   

3             40.790844        -73.976124         40.803349   

4             40.744085        -73.925023         40.761247   

...                 ...               ...               ...   

199995        40.739367        -73.987042         40.740297   

199996        40.736837        -73.984722         40.739620   

199997        40.756487        -73.986017         40.692588   

199998        40.725452        -73.997124         40.695415   

199999        40.720077        -73.984395         40.768793   



        dropoff_longitude  distance_km  

0              -73.999512     1.683323  

1              -73.994710     2.457590  

2              -73.962565     5.036377  

3              -73.965316     1.661683  

4              -73.9

In [11]:
dist = pd.DataFrame(dist)

In [12]:
df = pd.concat([df, dist['distance_km']], axis=1)

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [14]:
features = df[['passenger_count','distance_km']]

In [15]:
X = features
y = df['fare_amount']

In [16]:
lr = LinearRegression()

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=101)

In [18]:
lr.fit(X_train, y_train)

LinearRegression()

In [19]:
preds = lr.predict(X_test)

In [20]:
np.sqrt(mean_squared_error(y_true=y_test, y_pred=preds))

np.float64(9.827820337080789)

In [21]:
r2_score(y_true=y_test, y_pred=preds)

0.0005764797038434466

In [22]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [23]:
dtr = DecisionTreeRegressor()

In [24]:
dtr.fit(X_train, y_train)

DecisionTreeRegressor()

In [25]:
preds = dtr.predict(X_test)

In [26]:
np.sqrt(mean_squared_error(y_true=y_test, y_pred=preds))

np.float64(7.602472552728264)

In [27]:
r2_score(y_true=y_test, y_pred=preds)

0.40193984290398477

In [28]:
rfr = RandomForestRegressor()

In [29]:
rfr.fit(X_train, y_train)

RandomForestRegressor()

In [30]:
preds = rfr.predict(X_test)

In [31]:
np.sqrt(mean_squared_error(y_true=y_test, y_pred=preds))

np.float64(6.235405456632631)

In [32]:
r2_score(y_true=y_test, y_pred=preds)

0.5976865553724571

The Random Forest Regression model performs the best among the models i tested, with the lowest RMSE and the highest R² score.